# Curso de Big Data

## Trabajo Práctico 3

### Samuel Arispe, Gonzalo Ochoa y Facundo Valle Quintana

### Reglas de formato y presentación
- El trabajo debe estar bien comentado (utilizando #) para que el código sea fácil de entender por sus compañeros y profesores.

- El mismo debe ser completado en este Jupyter Notebook y entregado como tal, es decir en un archivo .ipynb


### Fecha de entrega:
Domingo 5 de noviembre a las 23:59hs

### Contenidos:


Analizar el problema del no reporte de ingresos en las encuestas de hogares y cómo eso afecta la medición de la tasa de pobreza. Se profundizará en el análisis de esta temática con los nuevos métodos vistos en clase para identificar individuos sin reporte de ingresos.

### Modalidad de entrega
- Al finalizar el trabajo práctico deben hacer un último <i>commit</i> en su repositorio de GitHub con el mensaje “Entrega final del tp”.

- Asegurense de haber creado una carpeta llamada TP3. Deben entregar el código (jupyter notebook).

- También deben completar el link de su repositorio -para que pueda ser clonado y corregido- en la siguiente google sheet: https://docs.google.com/spreadsheets/d/1v8Sv2TSzpK6mxjm3mmdnhmkk0qiyGFNj/edit#gid=1820558252

- La última versión en el repositorio es la que será evaluada. Por lo que es importante que:
    - No completen la google sheet hasta no haber terminado y estar seguros de que han hecho el <i>commit y push</i> a la versión final que quieren entregar. Debido a que se pueden tomar hasta 3 días de extensión a lo largo del curso, no se  corregirán sus tareas hasta no ver el link en la google sheet.\
    
    - No hagan nuevos <i>push</i> despues de haber entregado su versión final. Esto generaría confusión acerca de que versión es la que quieren que se les corrija.

### Reglas de formato y presentación

- El trabajo se debe entregar como Jupyter Notebook. Se espera una buena redacción en la resolución del práctico.

- Identifiquen claramente a qué inciso corresponde cada comando y, cuando sea necesario, usen bloques de texto para detallar sus respuestas.

- Importante: Todos los miembros del equipo deben haber hecho al menos un <i> commit </i> durante la realización del TP para asegurar que todos hayan aportado a su resolución.

## Parte I: Análisis de la base de hogares y cálculo de pobreza

Ahora que ya se han familiarizado con la Encuesta Permanente de Hogares (EPH) y
con la tabla de adulto equivalente, vamos a complejizar un poco la construcción del índice de pobreza. Lo calcularemos a nivel de hogares, con sus respectivos factores de expansión y realizaremos una limpieza de la base con mayor dedicación.
A continuación, complementamos el trabajo hecho en el TP2 usando la encuesta a
nivel hogares de la EPH.

#### 1. Exploren el diseño de registro de la base de hogar: a priori, ¿qué variables creen que pueden ser muy predictivas de pobreza y que sería muy útil incluir para perfeccionar el ejercicio del TP2?

Respuesta:

#### 2. Descarguen la base de microdatos de la EPH correspondiente al primer trimestre de 2023 (la base de hogares se llama usu hogar T123.xls). Importen los datos de la encuesta de hogar y, al igual que en el TP2, conserven sólo las observaciones que corresponden a los aglomerados de Ciudad Autónoma de Buenos Aires o del Gran Buenos Aires.

#### 3. Unan la tabla de la encuesta individual con la de la encuesta de hogar.

#### 4. Generen sus propias funciones para limpiar la base de datos o, si deciden utilizar funciones existentes en paquetes como numpy y pandas, mencionen cúales usarán y de qué paquetes son.

#### 5. Limpien la base de datos tomando criterios que hagan sentido, tanto para el tratamiento de valores faltantes, de outliers, como así también decidan qué variables categóricas y strings usarán y transfórmenlas de forma que haga sentido para los ejercicios siguientes. Justifiquen sus decisiones.

#### 6. Presenten estadísticas descriptivas de cinco variables de la encuesta de hogar que ustedes creen que pueden ser relevantes para predecir pobreza.

#### 7. Repitan el inciso 1.2.f del TP2 para construir la columna <i>adulto_equiv</i> y la columna <i>ad_equiv_hogar</i> (pueden utilizar su código del TP2).

#### 8. Repitan el inciso 1.3 y 1.4 del TP2 para dividir la base en dos dataframes donde: uno conserve las personas que no reportaron ITF (dataframe llamado <i> respondieron </i>) y otro conserve a las personas que no reportaron ITF (llamado <i> norespondieron </i>). Además, agreguen a la base respondieron una columna llamada <i>ingreso_necesario</i> que sea el producto de la canasta básica por <i>ad_equiv_hogar</i>.

#### 9. Agreguen a la base respondieron una columna llamada pobre, que tome valor1 si el ITF es menor al ingreso necesario que necesita esa familia y 0 en caso contrario.

#### 9. En el TP2 calcularon los individuos bajo la l´ınea de pobreza. Sin embargo, cuando se habla de pobreza el número más utilizado es el de la tasa de hogares bajo la línea de pobreza. Para calcularlo, utilicen una sola observación por hogar y sumen el ponderador PONDIH que permite expandir la muestra de la EPH al total de la población que representa. ¿Cúal es la tasa de hogares bajo la línea de pobreza para el GBA? ¿Se asemeja al que reporta el INDEC en sus informes? (link a informe: 
https://www.indec.gob.ar/uploads/informesdeprensa/eph_pobreza_09_2326FC0901C2.pdf)

## Parte II: Construcción de funciones

El objetivo de esta parte del trabajo es generar código que sea flexible y que esté modularizado (en funciones bien documentadas con docstrings). De esta forma, evitarán repetir código y podrán utilizarlo en distintos escenarios (como por ejemplo la Parte III de este TP y sus proyectos personales a futuro).

#### 1. Escriban una función, llamada evalua metodo, que reciba como argumentos un modelo y los datos de entrenamiento y prueba (<i>X train </i>, <i> y train </i> , <i>X test</i>, <i>y test</i>). La funcioón debe ajustar el modelo con los datos de entrenamiento y calcular las métricas que considere necesarias para esta problemática (de mínima, deben reportar la matriz de confusión, las curvas ROC y los valores de AUC y de accuracy score de cada método). El output de la función debe ser una colección con las métricas evaluadas.

#### 2. Escriban una función, llamada <i>cross_validation</i>, que realice validación cruzada con <i>k</i> iteraciones (<i>k</i>-fold CV), llamando a la función del inciso anterior en cada una, pero para las <i>k</i> distintas particiones. La función debe recibir como argumentos el modelo, el valor de <i>k</i> y un dataset (es decir, sólo X e y). Pueden ayudarse con la función KFold para generar las particiones necesarias.

Kfold : https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html

#### 3. Escriban una función, llamada <i>evalua_config</i> que reciba una lista de configuraciones de hiperparámetros (1) (los distintos valores a probar como hiperparámetros podrian codificarse en diccionarios de Python) y utilizando la función <i>cross_validation</i> obtenga el error (2) promedio para cada configuración. Finalmente, la función debe devolver la configuración que genere menor error (3).

Notas: 

(1) En <i>scikit-learn</i>, muchos métodos llaman penalty al método de regularización y <i>C</i> a la inversa del hiperparámetro <i>λ</i>.

(2) Utilicen la medición del error que prefieran. Una opción sería el Error Cuadrático Medio

(3) Consejo: cuanto más genérica construyan la función, luego podrá ser utilizada en más situaciones. Por ahora, la usaremos solo para buscar el <i>λ</i> óptimo cuando utilicemos regularización.

#### 4. Escriban una función llamada <i>evalua_multiples_metodos</i> que les permita implementar los siguiente métodos con los hiperparámetros que ustedes elijan. Para la regresión logística, asegúrense de que esta función utilice su función <i>evalua_config</i> para optimizar el <i>λ</i> de la regularización. Finalmente, el output de la función debe ser una tabla donde las columnas sean las métricas que hayan evaluado (las que hayan incluido en la función evalua metodo) y las filas sean los modelos (con su configuración de hiperparámetros asociada) que hayan corrido. Asegúrense de que la tabla incluya una columna con nombre del modelo y el valor de los hiperparámetros/configuración: (4)

- Regresión logística
- Análisis de discriminante lineal
- KNN

Notas: 

(4) Pista: para la regresión logística, cuando incluyan regularización observen que deberán correr la función evalua metodo dos veces. Una para optimizar los hiperparámetros (con un set de datos para train y otro para validación) y otra para obtener las métricas con el hiperparámetro óptimo (con un set de datos para train y otro para test).

## Parte III: Clasificación y regularización

El objetivo de esta parte del trabajo es nuevamente intentar predecir si una persona es o no pobre utilizando datos distintos al ingreso, dado que muchos hogares son reacios a responder cúanto ganan. Esta vez lo haremos con la base unida de las preguntas de la encuesta individual y la encuesta de hogar. A su vez, incluiremos ejercicios de regularización y de validación cruzada.

#### 1. Eliminen de ambas bases (<i>respondieron</i>, <i>norespondieron</i>) todas las variables relacionadas a ingresos (en el archivo Diseño de bases y estructura ver las categorías: ingresos de la ocupación principal de los asalariados, ingresos de la ocupación principal, ingresos de otras ocupaciones, ingreso total individual, ingresos no laborales, ingreso total familiar, ingreso per cápita familiar). Eliminen también las columnas <i>adulto_equiv</i>, <i>ad_equiv_hogar</i> e <i>ingreso_necesario</i>. Establezcan a la variable pobre como su variable dependiente (vector y). El resto de las variables serán las variables independientes (matriz X). Dependiendo de la funcióon que usen, no se olviden de agregar la columna de 1 cuando sea necesario.

Archivo de diseño de bases y estructura: https://www.indec.gob.ar/ftp/cuadros/menusuperior/eph/EPH_registro_1T2023.pdf

#### 2. Corran la función evalua múltiples metodos con la base respondieron. En los próximos incisos profundizaremos en la tarea de regularización, pero en este ejercicio prueben al menos un hiperparámetro para regularizar y al menos un valor de <i>λ</i>.

#### 3. Expliquen cómo elegirían <i>λ</i> por validación cruzada. Detallen por qué no usarían el conjunto de prueba (test) para su elección.

#### 4. En validación cruzada, ¿cúal es el problema de usar un <i>k</i> muy pequeño y uno muy grande? Cuando <i>k = n</i> (con <i>n</i> el número de muestras), ¿cúantas veces se estima el modelo?

#### 5. Realicen un barrido en <i>λ = 10^n </i> con <i>n</i> ∈ {−5,−4,−3 . . . , +4, +5} y utilicen 10 fold CV para elegir el <i>λ</i> óptimo en regresión logística con Ridge y con LASSO. ¿Qué <i>λ</i> seleccionó en cada caso? Generen box-plots mostrando la distribución del error de predicción para cada <i>λ</i>. Cada box debe corresponder a un valor de <i>λ</i> y contener como observaciones el error medio de validación para cada partición. Además, para la regularización LASSO, genere un box-plot similar, pero ahora graficando la proporción de variables ignoradas por el modelo en función de <i>λ</i>, es decir la proporción de variables para las cuales el coeficiente asociado es cero.

Box Plot: https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.boxplot.html

#### 6.  En el caso del valor óptimo de <i>λ</i> para LASSO encontrado en el inciso anterior, ¿qué variables fueron descartadas? ¿Son las que hubieran esperado? ¿Tiene relación con lo que respondieron en el inciso 1 de la Parte I?

#### 7. Elijan alguno de los modelos de regresión logística donde hayan probado distintos parametros de regularización y comenten: ¿Qué método de regularización funcionó mejor Ridge o LASSO? Comenten mencionando el error cuadrático medio (ECM).

#### 8. ¿Cúal de todos los métodos evaluados predice mejor? ¿Con qué hiperparámetros? Justifiquen detalladamente utilizando las medidas de precisión que conocen.

#### 9. Con el método que seleccionaron, predigan qué personas son pobres dentro de la base <i>norespondieron</i>. ¿Qué proporción de los hogares son pobres en esa submuestra?